# Decomposição LU

No ponto atual, conseguimos resolver um sistema linear, sobre algumas
condições. Temos um problema, no entanto, e se quisermos resolver
mais de um sistema linear, e não tivermos todas os vetores do lado direito de imediato?

Por exemplo, considere o seguinte algoritmo

1. Comece com $v_0 = (1,1,\dots,1)$, $k = 0$
2. Calcule $\mu_k = \Vert v_k\Vert$,
3. Calcule $w_k = v_k/\mu_k$,
4. Encontre $v_{k+1}$ resolvendo $Av_{k+1} = w_k$.
5. Faça $k = k + 1$ e volte ao ponto 2.

Precisamos resolver muitos sistemas com a mesma matriz $A$.
Ao usar a eliminação Gaussiana, precisamos aplicar as mesmas operações
em $w_k$, e da maneira que está isso não é possível.

No entanto, como já mencionamos antes, se tivermos os valores de $m_{ij}$,
podemos refazer essas operações.
Dessa maneira, seria vantajoso modificar o método para retornar também os
elementos $m_{ij}$ e aí para cada vetor $b$ dado, aplicamos essas operações
nele.

Em outras palavras, modificamos o método de modo que dado $A$, retorna $U$ e os
elementos $m_{2,1}, m_{3,1}, \dots, m_{n,n}$.

A partir disso, para resolver o sistema $Ax = b$, primeiro aplicamos
$m_{2,1}, \dots, m_{n,n}$ à $b$, obtendo $c$, e resolvemos $Ux = c$.

Não por acaso, os elementos $m_{2,1},\dots,m_{n,n}$ nos permitem definir uma matriz
$L$ dada por
$$L_{i,j} = \left\{\begin{array}{ll}
m_{i,j}, & i > j \\
1, & i = j \\
0, & i < j,
\end{array}\right.
$$
de modo que transformar $b$ em $c$ é simplesmente a resolução do sistema
$b = Lc$, ou seja $c = L^{-1}b$.

Mais ainda, transformar $A$ em $U$ é também aplicar estas operações, ou seja
$U = L^{-1}A$.

Daí, obtemos a seguinte relação $A = LU$, e um nome para esta modificação:

## Decomposição LU

In [49]:
function dec_lu(A; tol=1e-12)
    m,n = size(A)
    U = copy(A)
    L = eye(m,n)
    for j = 1:n
        if abs(U[j,j]) < tol
            error("U_{j,j} ≈ 0.0")
        end
        for i = j+1:m
            mij = U[i,j]/U[j,j]
            L[i,j] = mij
            U[i,j] = 0.0
            for k = j+1:n
                U[i,k] -= mij*U[j,k]
            end
        end
    end
    return L, U
end

dec_lu (generic function with 1 method)

In [26]:
A = [3.0 1 2; -1 2 1; 1 1 4]
L, U = dec_lu(A)

(
3x3 Array{Float64,2}:
  1.0       0.0       0.0
 -0.333333  1.0       0.0
  0.333333  0.285714  1.0,

3x3 Array{Float64,2}:
 3.0  1.0      2.0    
 0.0  2.33333  1.66667
 0.0  0.0      2.85714)

In [27]:
L*U

3x3 Array{Float64,2}:
  3.0  1.0  2.0
 -1.0  2.0  1.0
  1.0  1.0  4.0

In [28]:
A

3x3 Array{Float64,2}:
  3.0  1.0  2.0
 -1.0  2.0  1.0
  1.0  1.0  4.0

Agora, para resolver o sistema $Ax = b$, fazemos
$$ LUx = b, $$
e fazendo $Ux = c$, temos
$$ Lc = b \\
Ux = c $$

Isto é, resolvemos primeiro $Lc = b$ obtendo $c$, e depois
$Ux = c$, obtendo $x$.

Precisamos implementar a resolução de sistemas triangulares inferioriores.

In [29]:
function tri_inf(L, b)
    n = length(b)
    c = zeros(n)
    for i = 1:n
        c[i] = b[i]
        for j = i-1:-1:1
            c[i] -= L[i,j]*c[j]
        end
        c[i] = c[i]/L[i,i]
    end
    return c
end

tri_inf (generic function with 1 method)

In [30]:
b = A*ones(3)
c = tri_inf(L, b)

3-element Array{Float64,1}:
 6.0    
 4.0    
 2.85714

In [31]:
# tri_sup.jl foi feito pelo aluno
include("tri_sup.jl")
x = tri_sup(U, c)

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

## Pivoteamento

In [32]:
A = [0.0 1 1; 2 1 2; -1 -1 4]
b = A*ones(3)

3-element Array{Float64,1}:
 2.0
 5.0
 2.0

In [3]:
det(A)

-11.0

$A$ é não singular, no entanto o método dado não funciona

In [50]:
function elim_gauss(A, b; tol=1e-12)
    m,n = size(A)
    U = copy(A)
    c = copy(b)
    for j = 1:n
        if abs(U[j,j]) < tol
            error("U_{j,j} ≈ 0.0")
        end
        for i = j+1:m
            mij = U[i,j]/U[j,j]
            U[i,j] = 0.0
            for k = j+1:n
                U[i,k] -= mij*U[j,k]
            end
            c[i] -= mij*c[j]
        end
    end
    return U, c
end

elim_gauss (generic function with 2 methods)

In [35]:
U, c = elim_gauss(A, b)

LoadError: LoadError: U_{j,j} ≈ 0.0
while loading In[35], in expression starting on line 1

Mas veja que o sistema equivalente abaixo tem solução pelo algoritmo:

In [36]:
A[[1;3],:] = A[[3;1],:]
A

3x3 Array{Float64,2}:
 -1.0  -1.0  4.0
  2.0   1.0  2.0
  0.0   1.0  1.0

In [37]:
b[[1;3]] = b[[3;1]]
b

3-element Array{Float64,1}:
 2.0
 5.0
 2.0

In [38]:
U, c = elim_gauss(A, b)

(
3x3 Array{Float64,2}:
 -1.0  -1.0   4.0
  0.0  -1.0  10.0
  0.0   0.0  11.0,

[2.0,9.0,11.0])

In [39]:
tri_sup(U, c)

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

Uma estratégia então seria adicionar a operação elementar de troca de linhas dentro da eliminação.

O número escolhido na coluna é chamado de pivô, e faremos a escolha da linha
$$ \hat{i} = \arg\max_{i = j,\dots,m} \left\vert\ a_{ij}\right\vert, $$
isto é, aquele elemento de maior valor em módulo. O motivo dessa escolha será deixado claro em breve.

In [51]:
function elim_gauss_pivot(A, b; tol=1e-12)
    m,n = size(A)
    U = copy(A)
    c = copy(b)
    for j = 1:n
        i_pivo, pivo = j, abs(U[j,j])
        for i = j+1:m
            if abs(U[i,j]) > pivo
                pivo = abs(U[i,j])
                i_pivo = i
            end
        end
        if i_pivo != j
            U[[i_pivo; j],:] = U[[j; i_pivo],:]
            c[[i_pivo; j]] = c[[j; i_pivo]]
        end
        if abs(U[j,j]) < tol
            error("U_{j,j} ≈ 0.0")
        end
        for i = j+1:m
            mij = U[i,j]/U[j,j]
            U[i,j] = 0.0
            for k = j+1:n
                U[i,k] -= mij*U[j,k]
            end
            c[i] -= mij*c[j]
        end
    end
    return U, c
end

elim_gauss_pivot (generic function with 1 method)

In [43]:
A = [0.0 1 1; 2 1 2; -1 -1 4]
b = A*ones(3)
U, c = elim_gauss_pivot(A, b)

(
3x3 Array{Float64,2}:
 2.0  1.0  2.0
 0.0  1.0  1.0
 0.0  0.0  5.5,

[5.0,2.0,5.5])

In [44]:
tri_sup(U, c)

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

Agora, queremos fazer essa mesma modificação na decomposição LU.
No entanto, como a mudança em $b$ é feita a posteriori, o resultado
é um pouco mais complicado.
Se ainda quisermos retornar apenas $L$ e $U$, o resultado deveria vir
com as linhas de $L$ e $U$ "destrocadas", isto é, $L$ e $U$ não seriam
triangulares.

A outra alternativa, preferida aqui, é retornar uma matriz a mais, $P$
de permutação das linhas, que é a matriz identidade com as permutações
de linha que seriam feitas na $A$. A partir daí, obtemos $P$ tal que

$$ PA = LU. $$

In [58]:
function dec_lu_pivot(A; tol=1e-12)
    m,n = size(A)
    U = copy(A)
    L = eye(m,n)
    P = eye(Int, m)
    for j = 1:n
        i_pivo, pivo = j, abs(U[j,j])
        for i = j+1:m
            if abs(U[i,j]) > pivo
                pivo = abs(U[i,j])
                i_pivo = i
            end
        end
        if pivo < tol
            error("pivo ≈ 0.0")
        end
        if i_pivo != j
            U[[i_pivo; j],:] = U[[j; i_pivo],:]
            P[[i_pivo; j],:] = P[[j; i_pivo],:]
        end
        for i = j+1:m
            
            mij = U[i,j]/U[j,j]
            L[i,j] = mij
            U[i,j] = 0.0
            for k = j+1:n
                U[i,k] -= mij*U[j,k]
            end
        end
    end
    return L, U, P
end

dec_lu_pivot (generic function with 1 method)

In [59]:
L, U, P = dec_lu_pivot(A)

(
3x3 Array{Float64,2}:
  1.0   0.0  0.0
  0.0   1.0  0.0
 -0.5  -0.5  1.0,

3x3 Array{Float64,2}:
 2.0  1.0  2.0
 0.0  1.0  1.0
 0.0  0.0  5.5,

3x3 Array{Int64,2}:
 0  1  0
 1  0  0
 0  0  1)

In [60]:
L*U - P*A

3x3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

Veja que $P$ tem uma propriedade interessante:

In [61]:
P'*P

3x3 Array{Int64,2}:
 1  0  0
 0  1  0
 0  0  1

Sua inversa é sua transposta. Matriz quadrados com essa propriedade são ditas
**ortogonais**, e $P$ é um caso ainda mais especial dito **matriz de permutação de linhas**.

A resolução do sistema $Ax = b$ agora inicia multiplicando-se por $P$.
$$ PAx = Pb. $$
Daí, prosseguimos como antes.
$$ PAx = Pb \qquad \Rightarrow \qquad
LUx = Pb \qquad \Rightarrow \qquad
\left\{\begin{array}{l}
Lc = Pb \\
Ux = c
\end{array}\right.
$$

In [76]:
c = tri_inf(L, P*b)
x = tri_sup(U, c)

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

## Complexidade de LU

As operações de LU são as mesmas que a eliminação Gaussiana,
sem as operações de transformar $b$ em $c$.
Note que as soluções triangulares são $\mathcal{O}(n^2)$.

Uma dificuldade, no entanto é o acréscimo de memória, que é essencialmente
de uma matriz de zeros. Para aplicações "sérias", precisamos nos atentar a
esse fato. Uma maneira de fazer isso é utilizar a própria matriz $A$ para
armazenar $LU$ e retornar um vetor $P$ de índices indicando as mudanças
de linhas.

Por exemplo, ao invés de termos

In [69]:
L, U, P = dec_lu_pivot(A)

(
3x3 Array{Float64,2}:
  1.0   0.0  0.0
  0.0   1.0  0.0
 -0.5  -0.5  1.0,

3x3 Array{Float64,2}:
 2.0  1.0  2.0
 0.0  1.0  1.0
 0.0  0.0  5.5,

3x3 Array{Int64,2}:
 0  1  0
 1  0  0
 0  0  1)

Guardamos da seguinte maneira:

In [73]:
LU = tril(L,-1) + U

3x3 Array{Float64,2}:
  2.0   1.0  2.0
  0.0   1.0  1.0
 -0.5  -0.5  5.5

In [74]:
PI = [findfirst(P[i,:] .== 1) for i = 1:3]

3-element Array{Any,1}:
 2
 1
 3

A entrada da função será $A$, e a saída será $PI$, mas a **única** coisa criada é o vetor $PI$. $A$ deve ser substituída por $LU$.

In [89]:
function dec_lu_pivot!(A; tol=1e-12)
    m,n = size(A)
    P = collect(1:m)
    for j = 1:n
        i_pivo, pivo = j, abs(A[j,j])
        for i = j+1:m
            if abs(A[P[i],j]) > pivo
                pivo = abs(A[P[i],j])
                i_pivo = i
            end
        end
        if pivo < tol
            error("pivo ≈ 0.0")
        end
        if i_pivo != j
            P[[i_pivo; j]] = P[[j; i_pivo]]
        end
        pj = P[j]
        for i = j+1:m
            pi = P[i]
            mij = A[pi,j]/A[pj,j]
            A[pi,j] = mij
            for k = j+1:n
                A[pi,k] -= mij*A[pj,k]
            end
        end
    end
    return P
end

dec_lu_pivot! (generic function with 1 method)

In [92]:
A = [0.0 1 1; 2 1 2; -1 -1 4]

3x3 Array{Float64,2}:
  0.0   1.0  1.0
  2.0   1.0  2.0
 -1.0  -1.0  4.0

In [93]:
P = dec_lu_pivot!(A)

3-element Array{Int64,1}:
 2
 1
 3

In [94]:
A

3x3 Array{Float64,2}:
  0.0   1.0  1.0
  2.0   1.0  2.0
 -0.5  -0.5  5.5

Compare com L e U:

In [95]:
L, U

(
3x3 Array{Float64,2}:
  1.0   0.0  0.0
  0.0   1.0  0.0
 -0.5  -0.5  1.0,

3x3 Array{Float64,2}:
 2.0  1.0  2.0
 0.0  1.0  1.0
 0.0  0.0  5.5)

A resolução do sistema $Ax = b$ com essa modificação agora é mais complicada.